<a href="https://colab.research.google.com/github/GustavoCruzzz/ModeloAnaliseViabilidadedeProjetos/blob/main/ModeloAnaliseViabilidadedeProjetos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import numpy as np
import joblib
import os

In [ ]:
def train_or_predict(new_projects):
  model_path = 'logistic_model.joblib'
  if os.path.exists(model_path):
    model = joblib.load(model_path)
    scaler = joblib.load(model_path.replace(".joblib", "_scaler.joblib"))
  else:
    df_projects = pd.read_csv("projects_data.csv")

    x = df_projects[["investment", "expected_return", "impact_score"]]
    y = df_projects["viability"]


    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(x)

    X_train, x_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

    model = LogisticRegression()
    model.fit(X_train, y_train)

    y_pred = model.predict(x_test)
    report = classification_report(y_test, y_pred, output_dict=True)

    joblib.dump(model, model_path)
    joblib.dump(scaler, model_path.replace(".joblib", "_scaler.joblib"))
    joblib.dump(report, model_path.replace(".joblib", "_metrics.joblib"))

    if new_projects:
      df_new_projects = pd.DataFrame(new_projects)

      X_new_scaled = scaler.transform(df_new_projects)

      predictions = model.predict(X_new_scaled)

      probabilities = model.predict_proba(X_new_scaled)[:, 1]
      df_new_projects["probability"] = probabilities
      df_new_projects["viability"] = predictions

      return df_new_projects, joblib.load(model_path.replace(".joblib", "_metrics.joblib"))

      return None, joblib.load(model_path.replace(".joblib", "_metrics.joblib"))


In [ ]:
new_projects = [{"investment": 13000, "expected_return": 69000, "impact_score": 7}]

predictions, metrics = train_or_predict(new_projects)

if predictions is not None:
  print("\nNovos Projetos e Viabilidade")
  print(predictions)

  print("\nMétricas do Modelo:")
  print(metrics)


Novos Projetos e Viabilidade
   investment  expected_return  impact_score  probability  viability
0       13000            69000             7     0.729334          1

Métricas do Modelo:
{'0': {'precision': 0.8768472906403941, 'recall': 0.9128205128205128, 'f1-score': 0.8944723618090452, 'support': 195.0}, '1': {'precision': 0.8247422680412371, 'recall': 0.7619047619047619, 'f1-score': 0.7920792079207921, 'support': 105.0}, 'accuracy': 0.86, 'macro avg': {'precision': 0.8507947793408156, 'recall': 0.8373626373626373, 'f1-score': 0.8432757848649186, 'support': 300.0}, 'weighted avg': {'precision': 0.8586105327306891, 'recall': 0.86, 'f1-score': 0.8586347579481568, 'support': 300.0}}
